In [2]:
import requests
import json
from urllib import parse
import pprint

api_key = "RGAPI-85747cdd-d883-496f-b901-d46ffa0d29e6"
REQUEST_HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36 OPR/114.0.0.0 (Edition GX-CN)",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key,
}

In [20]:
userNickname = "Hide on bush"
tagLine = "KR1"
encodedName = parse.quote(userNickname)
url = f"https://asia.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{encodedName}/{tagLine}"
player_id = requests.get(url, headers=REQUEST_HEADERS).json()
pprint.pprint(player_id)

{'gameName': 'Hide on bush',
 'puuid': 'AsmHmmVr9ymowYq-ZSwgQZLUNejxdulG92SbAfH6oFC7x15fgyg6u2PbAG_kkufRZY3iE28ZWWkUHw',
 'tagLine': 'KR1'}


In [24]:
puuid = player_id["puuid"]
url = f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{puuid}"
player = requests.get(url, headers=REQUEST_HEADERS).json()
pprint.pprint(player)

{'accountId': '1CjeGMQANYZB7KnwICRqfoNKqZ6xLRBpjcZ_itsqLtPs',
 'id': 'OM14Njy73Vm8sQtW0kYQE3bCmesqZ_HnwI0IG6hlqT_cdQ',
 'profileIconId': 6,
 'puuid': 'AsmHmmVr9ymowYq-ZSwgQZLUNejxdulG92SbAfH6oFC7x15fgyg6u2PbAG_kkufRZY3iE28ZWWkUHw',
 'revisionDate': 1736012649926,
 'summonerLevel': 789}


In [25]:
playerInfo = requests.get(
    "https://kr.api.riotgames.com/lol/league/v4/entries/by-summoner/" + player["id"],
    headers=REQUEST_HEADERS,
).json()
pprint.pprint(playerInfo)

[{'freshBlood': True,
  'hotStreak': False,
  'inactive': False,
  'leagueId': 'b6a02cb5-0bde-3378-b3e1-af92b837db56',
  'leaguePoints': 118,
  'losses': 4,
  'queueType': 'RANKED_SOLO_5x5',
  'rank': 'I',
  'summonerId': 'OM14Njy73Vm8sQtW0kYQE3bCmesqZ_HnwI0IG6hlqT_cdQ',
  'tier': 'MASTER',
  'veteran': False,
  'wins': 24}]


In [26]:
n_wins = playerInfo[0]["wins"]
n_losses = playerInfo[0]["losses"]
n_total = n_wins + n_losses
print(n_wins)
print(n_losses)
print(n_total)

r = n_total // 100
other = n_total % 100

all_gamesID = []
for i in range(r + 1):
    start = i * 100
    if i != r:
        tmp_gamesID = requests.get(
            f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{player['puuid']}/ids?type=ranked&start={start}&count={100}",
            headers=REQUEST_HEADERS,
        ).json()
    else:
        tmp_gamesID = requests.get(
            f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{player['puuid']}/ids?type=ranked&start={start}&count={other}",
            headers=REQUEST_HEADERS,
        ).json()

    all_gamesID.extend(tmp_gamesID)

24
4
28


In [27]:
gameInfo = requests.get(
    "https://asia.api.riotgames.com/lol/match/v5/matches/" + all_gamesID[0],
    headers=REQUEST_HEADERS,
).json()
pprint.pprint(gameInfo)

{'info': {'endOfGameResult': 'GameComplete',
          'gameCreation': 1736011452764,
          'gameDuration': 1179,
          'gameEndTimestamp': 1736012648421,
          'gameId': 7456171924,
          'gameMode': 'CLASSIC',
          'gameName': 'teambuilder-match-7456171924',
          'gameStartTimestamp': 1736011468788,
          'gameType': 'MATCHED_GAME',
          'gameVersion': '14.24.644.2327',
          'mapId': 11,
          'participants': [{'allInPings': 0,
                            'assistMePings': 0,
                            'assists': 9,
                            'baronKills': 0,
                            'basicPings': 0,
                            'bountyLevel': 2,
                            'challenges': {'12AssistStreakCount': 0,
                                           'HealFromMapSources': 220,
                                           'InfernalScalePickup': 0,
                                           'SWARM_DefeatAatrox': 0,
                    

In [14]:
def get_champion_data():
    url = "http://ddragon.leagueoflegends.com/cdn/14.24.1/data/ko_KR/champion.json"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code, response.text)
        return None

champion_data = get_champion_data()
print(champion_data)

{'type': 'champion', 'format': 'standAloneComplex', 'version': '14.24.1', 'data': {'Aatrox': {'version': '14.24.1', 'id': 'Aatrox', 'key': '266', 'name': '아트록스', 'title': '다르킨의 검', 'blurb': '한때는 공허에 맞서 싸웠던 슈리마의 명예로운 수호자 아트록스와 그의 종족은 결국 공허보다 위험한 존재가 되어 룬테라의 존속을 위협했지만, 교활한 필멸자의 마법에 속아넘어가 패배하게 되었다. 수백 년에 걸친 봉인 끝에, 아트록스는 자신의 정기가 깃든 마법 무기를 휘두르는 어리석은 자들을 타락시키고 육신을 바꾸는 것으로 다시 한번 자유의 길을 찾아내었다. 이제 이전의 잔혹한 모습을 닮은 육체를 차지한 아트록스는 세상의 종말과 오랫동안 기다려온 복수를...', 'info': {'attack': 8, 'defense': 4, 'magic': 3, 'difficulty': 4}, 'image': {'full': 'Aatrox.png', 'sprite': 'champion0.png', 'group': 'champion', 'x': 0, 'y': 0, 'w': 48, 'h': 48}, 'tags': ['Fighter'], 'partype': '피의 샘', 'stats': {'hp': 650, 'hpperlevel': 114, 'mp': 0, 'mpperlevel': 0, 'movespeed': 345, 'armor': 38, 'armorperlevel': 4.8, 'spellblock': 32, 'spellblockperlevel': 2.05, 'attackrange': 175, 'hpregen': 3, 'hpregenperlevel': 0.5, 'mpregen': 0, 'mpregenperlevel': 0, 'crit': 0, 'critperlevel': 0, 'attackdamage': 60, 'attackdamageperlevel'